In [1]:
# Data to train on (all works of shakesphere combined into one text file)
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-04 12:15:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-07-04 12:15:44 (10.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r') as file:
    text = file.read()

In [2]:
print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [3]:
# extract all unique chars from the entire corpus
all_chars = sorted(list(set(text)))
print("".join(all_chars))
len(all_chars)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [4]:
# creating a tokenizer at char level

stoi = {char:i for i, char in enumerate(all_chars)}
itos = {i:char for i, char in enumerate(all_chars)}

encode = lambda x: [stoi[char] for char in x]
decode = lambda x: ''.join([itos[i] for i in x])

In [5]:
decode(encode("you are a good boy"))

'you are a good boy'

In [6]:
import torch

encoded_text = torch.tensor(encode(text), dtype=torch.long)
print(encoded_text.shape, encoded_text.dtype)

torch.Size([1115394]) torch.int64


In [7]:
encoded_text[:1000]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [8]:
# split corpus into train and validation sets

split_n_idx = int(0.9 * len(text))
train_data = encoded_text[:split_n_idx]
val_data = encoded_text[split_n_idx:]

print(len(train_data), len(val_data), len(train_data)+len(val_data))

1003854 111540 1115394


In [9]:
# context length declared here
# bc the transformer cant take the full sequence, it would bee too long and computationally infeasible for large corpus bc of the n^2 attention mechanism whee n is the num of token in the sequence
# so we declare a smaller batch to be the input per training sample

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(len(x)):
    print(f"if input is {x[:i+1]} then output should be {y[i]}")

if input is tensor([18]) then output should be 47
if input is tensor([18, 47]) then output should be 56
if input is tensor([18, 47, 56]) then output should be 57
if input is tensor([18, 47, 56, 57]) then output should be 58
if input is tensor([18, 47, 56, 57, 58]) then output should be 1
if input is tensor([18, 47, 56, 57, 58,  1]) then output should be 15
if input is tensor([18, 47, 56, 57, 58,  1, 15]) then output should be 47
if input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then output should be 58


In [11]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

In [12]:
def get_batch(split: str, batch_size: int):
    data = train_data if split.lower() == "train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [13]:
xb, yb = get_batch("train", batch_size=4)
print(xb.shape, yb.shape)
print(xb)
print(yb)

for i in range(batch_size):
    for j in range(block_size):
        xi = xb[i, :j+1]
        yo = yb[i, j]
        print(f"Given {xi} target is: {yo}")

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
Given tensor([24]) target is: 43
Given tensor([24, 43]) target is: 58
Given tensor([24, 43, 58]) target is: 5
Given tensor([24, 43, 58,  5]) target is: 57
Given tensor([24, 43, 58,  5, 57]) target is: 1
Given tensor([24, 43, 58,  5, 57,  1]) target is: 46
Given tensor([24, 43, 58,  5, 57,  1, 46]) target is: 43
Given tensor([24, 43, 58,  5, 57,  1, 46, 43]) target is: 39
Given tensor([44]) target is: 53
Given tensor([44, 53]) target is: 56
Given tensor([44, 53, 56]) target is: 1
Given tensor([44, 53, 56,  1]) target is: 58
Given tensor([44, 53, 56,  1, 58]) target is: 46
Given tensor([44, 53, 56,  1, 58, 46]) target 

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)

            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [15]:
model = BigramLanguageModel(65)
logits, loss = model(xb, yb)
print(logits.shape, loss)

torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)


In [16]:
idx = torch.zeros((1, 1), dtype=torch.long)
result = model.generate(idx, max_new_tokens=50)

print(result)
print(decode(result[0].tolist()))

tensor([[ 0, 31, 23, 21, 41, 24, 32, 11, 13, 41, 17, 24, 25, 53, 32, 40, 60, 38,
         60,  1, 15, 12, 52, 55,  7, 29, 17,  9,  9, 10, 15, 22, 55, 49, 27, 23,
         20,  7, 55, 11, 10, 50, 39,  2, 53, 47, 63, 61, 49, 20, 48]])

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHj


In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [23]:
batch_size = 32

for step in range(10000):
    xb, yb = get_batch('train', batch_size)

    logits, loss = model(xb, targets=yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
    

2.45680570602417


In [27]:
idx = torch.zeros((1, 1), dtype=torch.long)
result = model.generate(idx, max_new_tokens=500)

print(decode(result[0].tolist()))


UClwanovead her t t APAUESLANEd titrake! GLTathed s,
RI lk.
Anordenare pine thal wh,
T:


eve,


Wheathas wouns Ply
MI lif corndiver ve. he thoutasen at, ond tren: ibrmesanouse hat rait ge kfals.
Ge Goowhe.
Mears is ise henel t gherdf meve.
OKIVEENG ol by t?
ANNGr uenowhisstee be
G bll ICleer sly f gs and tougr come te bue misein aip:
MBRGllinfre e pe mfas ale Fr?
CEro nd, dare is th we we wes?
DUS:
N HEdigen's y ple th mest r w be prm an sll ck y h ind, athoulkeavo An us ding bang aulcal ppe th


In [88]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [89]:
xbow = torch.zeros((B, T, C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)


In [90]:
x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [91]:
torch.manual_seed(42)

a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(f"{a}\n{b}\n{c}")

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [92]:
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [93]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = wei.softmax(-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)
xbow3[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [99]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

wei = q @ k.transpose(-2, -1) # (B, T, T)
wei = wei * head_size**-0.5 # normalization so the wei values are evenly diffused instead of having too high of a value in one node which will make the softmax "spiky"(focused on a single node, instead of ~evenly across all)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = wei.softmax(-1)

wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5150, 0.4850, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3342, 0.2973, 0.3686, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2330, 0.1956, 0.2701, 0.3013, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2028, 0.2231, 0.1988, 0.1965, 0.1788, 0.0000, 0.0000, 0.0000],
        [0.1514, 0.1827, 0.1263, 0.1103, 0.1483, 0.2809, 0.0000, 0.0000],
        [0.1438, 0.1328, 0.1517, 0.1577, 0.1505, 0.1194, 0.1442, 0.0000],
        [0.1049, 0.1558, 0.0818, 0.0683, 0.0813, 0.2506, 0.1041, 0.1533]],
       grad_fn=<SelectBackward0>)

In [98]:
v = value(x)
xbow4 = wei @ v
wei.shape, v.shape, xbow4.shape, v[0], wei[0]

(torch.Size([4, 8, 8]),
 torch.Size([4, 8, 16]),
 torch.Size([4, 8, 16]),
 tensor([[ 0.0660,  0.0865, -0.0022, -0.0979,  0.0494, -0.0847, -0.1617, -0.0495,
           0.1284,  0.1332,  0.0091,  0.0597,  0.1579, -0.0382,  0.0418, -0.0894],
         [-0.5969,  0.1531, -0.4713, -0.3066,  0.6378,  0.4022, -0.3253,  0.2137,
          -0.0749,  0.3800,  0.3823, -0.2537,  0.3899, -0.0850, -0.3499, -0.4370],
         [ 0.3468,  0.2170,  0.1132, -0.2115, -0.0160, -0.3525, -0.3953, -0.2007,
           0.3985,  0.2972, -0.0700,  0.2409,  0.3679, -0.0912,  0.2126, -0.1533],
         [ 0.5402,  0.3231,  0.1842, -0.3105, -0.0421, -0.5434, -0.5874, -0.3089,
           0.6036,  0.4377, -0.1165,  0.3708,  0.5441, -0.1352,  0.3307, -0.2190],
         [ 0.7093, -0.2305,  0.5855,  0.4262, -0.8141, -0.4590,  0.4793, -0.2418,
           0.0326, -0.5338, -0.4784,  0.2866, -0.5577,  0.1233,  0.4144,  0.5839],
         [-0.9490, -0.3276, -0.4496,  0.2398,  0.3522,  0.8613,  0.5735,  0.4826,
          -0.7818, 

In [102]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [103]:
tril[:T, :T]

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

### GPT from scratch from memory

In [34]:
import torch


ctx_len = 16
batch_size = 4
n_embed = 32
head_size = 16

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# Open data file
with open("input.txt", "r") as file:
    text = file.read()

# calculate vocab and vocab_size
unique_chars = sorted(list(set(text)))
vocab = unique_chars
vocab_size = len(unique_chars)

# make tokenizer with encode and decode functions
stoi = {char: i for i, char in enumerate(vocab)}
itos = {i: char for i, char in enumerate(vocab)}

encode = lambda prompt: [stoi[char] for char in prompt]
decode = lambda tokens: "".join([itos[i] for i in tokens])

In [26]:
# encode and split text into train and validation sets
encoded_text = torch.tensor(encode(text))
split_idx = int(0.9 * len(encoded_text))
train_data = encoded_text[:split_idx]
val_data = encoded_text[split_idx:]

In [27]:
len(train_data), len(val_data), len(train_data)+len(val_data) == len(encoded_text)

(1003854, 111540, True)

In [38]:
def batch_data():
    sample_idxs = torch.randint(len(train_data) - ctx_len, (batch_size,))
    input_data = torch.stack([train_data[idx: idx+ctx_len] for idx in sample_idxs])
    expected_data = torch.stack([train_data[idx+1: idx+ctx_len+1] for idx in sample_idxs])
    
    input_data.to(device)
    expected_data.to(device)
    
    return input_data, expected_data

In [51]:
x, y = batch_data()
x.shape

torch.Size([4, 16])

In [22]:
for i in sample_idxs:
    print(i)
    print(train_data[i: i+ctx_len])
    print(":"*100)

tensor(502859)
[47, 57, 57, 1, 41, 53, 52, 57, 59, 51, 43, 10, 1, 58, 46, 43]
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
tensor(645403)
[53, 52, 42, 43, 56, 1, 47, 52, 1, 43, 62, 58, 56, 43, 51, 43]
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
tensor(597081)
[57, 1, 54, 47, 43, 56, 41, 47, 52, 45, 1, 58, 39, 50, 53, 52]
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
tensor(148360)
[43, 1, 56, 53, 59, 45, 46, 6, 1, 59, 52, 57, 61, 39, 63, 39]
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
